In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import re
import string
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import svm
from sklearn.model_selection import train_test_split


In [2]:
tweets = (open("./../Dataset/train/us_train.text", "r").readlines())

In [3]:
allTweets = pd.DataFrame(tweets, columns=['tweets'])


In [4]:
stopword = nltk.corpus.stopwords.words('english')
stopword += ['yr', 'year', 'woman', 'man', 'girl','boy','one', 'two', 'sixteen', 'yearold', 'fu', 'weeks', 'week',
        'treatment', 'associated', 'patients', 'may','day', 'case','old']
ps = PorterStemmer()


In [5]:
def clean_text(text):
    text = re.sub(r'@user','',text)
    text  = "".join([char.lower() for char in text if char not in string.punctuation])
    text  = "".join([char.lower() for char in text if char not in ['…','\n']])
    text = re.sub('[0-9]+', '', text)
    text = ''.join(i for i in text if ord(i)<128)
    text = re.sub('\s+', ' ', text)
    text = word_tokenize(text)    
    for i in range(len(text)):
        word = text[i]
        newword = ""
        prev = word[0]
        for j in range(1,len(word)):
            if word[j] == prev:
                pass
            else:
                newword += word[j]
                prev = word[j]
        text[i] = newword

    text = [word for word in text if word not in stopword]
    #text = [str(TextBlob(word).correct()) for word in text]
    text = [word for word in text if word not in stopword]
    text = [ps.stem(word) for word in text]
    text = [word for word in text if 10>len(word)>2]
    for word in text:
        if len(word)<=2:
            print(word)
    return text

In [6]:
vectorizer = CountVectorizer(analyzer=clean_text)
countVector = vectorizer.fit_transform(allTweets['tweets'][:100000])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

100000 Number of tweets has 49862 words


In [19]:
tfidf_transformer = TfidfTransformer()
normalized_tdm = tfidf_transformer.fit_transform(countVector)

In [7]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [8]:
# countVector = countVector.astype('int8')

In [9]:
# count_vect_df = pd.DataFrame(countVector.toarray(), columns=vectorizer.get_feature_names())
# count_vect_df.head()



/home/karan/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aba,abab,ababayan,ababi,abachi,abacimino,abad,abaday,abado,abag,...,zusa,zvibe,zvsf,zya,zyboz,zyc,zychair,zyd,zyeah,zyr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
emoji = (open("./../Dataset/train/us_train.labels", "r").readlines())

In [22]:
emoji = emoji[:100000]

In [23]:
for i in tqdm(range(len(emoji))):
    emoji[i] = int(emoji[i][0])

100%|██████████| 100000/100000 [00:00<00:00, 1318511.58it/s]


In [24]:
emoji = (np.array(emoji)).astype('int8')

In [25]:
SVM = svm.SVC(kernel='linear')

In [15]:
final_array = count_vect_df.values

In [26]:
SVM.fit(normalized_tdm,emoji)

SVC(kernel='linear')

In [28]:
predictions = SVM.predict(normalized_tdm)
print(accuracy_score(predictions,emoji))